<a href="https://colab.research.google.com/github/Int-Group-44/flowers-102/blob/main/OxfordFlowers102CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras import layers
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
#device_name = tf.test.gpu_device_name()
#if device_name != '/device:GPU:0':
#  raise SystemError('GPU device not found')
#print('Found GPU at: {}'.format(device_name))

dataset, dataset_info = tfds.load('oxford_flowers102', with_info=True, as_supervised=True)
dataset_info
test_set, training_set, validation_set = dataset['test'], dataset['train'], dataset['validation']

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Shuffling /root/tensorflow_datasets/oxford_flowers102/2.1.1.incompleteHVH09I/oxford_flowers102-test.tfrecord*.…

Generating validation examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/oxford_flowers102/2.1.1.incompleteHVH09I/oxford_flowers102-validation.tfre…

Dataset oxford_flowers102 downloaded and prepared to /root/tensorflow_datasets/oxford_flowers102/2.1.1. Subsequent calls will reuse this data.


In [2]:
import tensorflow as tf
tf.config.experimental.list_physical_devices('GPU')
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
  except RuntimeError as e:
    print(e)

print(gpus)

Physical devices cannot be modified after being initialized
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


Importing TensorFlow 

In [3]:
num_classes = dataset_info.features['label'].num_classes
num_training_examples = 1020
num_validation_examples = 1020

In [4]:
IMAGE_RES = 224

def format_image(image, label):
    image = tf.image.resize(image, (IMAGE_RES, IMAGE_RES))/255.0
    #label = tf.one_hot(label, depth=num_classes)
    return image, label
BATCH_SIZE = 16
train_batches = training_set.shuffle(num_training_examples//4).map(format_image).batch(BATCH_SIZE).prefetch(1)
validation_batches = validation_set.shuffle(num_validation_examples//4).map(format_image).batch(BATCH_SIZE).prefetch(1)
test_batches = test_set.map(format_image).batch(BATCH_SIZE).prefetch(1)


In [5]:
def make_model(input_shape, num_classes):
  model = tf.keras.Sequential()
  model.add(layers.RandomFlip("horizontal_and_vertical", input_shape=input_shape))
  model.add(layers.RandomRotation(20))
  model.add(layers.RandomZoom(0.2))
  model.add(layers.Conv2D(32, 3, padding='same', activation='relu'))
  model.add(layers.BatchNormalization())
  model.add(layers.Conv2D(32, 3, padding='same', activation='relu'))
  model.add(layers.BatchNormalization())
  model.add(layers.MaxPooling2D())
  model.add(layers.Conv2D(64, 3, padding='same', activation='relu'))
  model.add(layers.BatchNormalization())
  model.add(layers.Conv2D(64, 3, padding='same', activation='relu'))
  model.add(layers.BatchNormalization())
  model.add(layers.MaxPooling2D())
  model.add(layers.Conv2D(128, 3, padding='same', activation='relu'))
  model.add(layers.BatchNormalization())
  model.add(layers.Conv2D(128, 3, padding='same', activation='relu'))
  model.add(layers.BatchNormalization())
  model.add(layers.MaxPooling2D())
  model.add(layers.Conv2D(256, 3, padding='same', activation='relu'))
  model.add(layers.BatchNormalization())
  model.add(layers.Conv2D(256, 3, padding='same', activation='relu'))
  model.add(layers.BatchNormalization())
  model.add(layers.MaxPooling2D())
  model.add(layers.Conv2D(512, 3, padding='same', activation='relu'))
  model.add(layers.BatchNormalization())
  model.add(layers.Conv2D(512, 3, padding='same', activation='relu'))
  model.add(layers.BatchNormalization())
  model.add(layers.MaxPooling2D())
  model.add(layers.Flatten())
  model.add(layers.Dense(1024, activation='relu'))
  model.add(layers.Dropout(0.5))
  model.add(layers.Dense(512, activation='relu'))
  model.add(layers.Dropout(0.5))
  model.add(layers.Dense(256, activation='relu'))
  model.add(layers.Dropout(0.5))
  model.add(layers.Dense(num_classes))
  return model


In [9]:
tf.keras.backend.clear_session()

In [11]:
model = make_model(input_shape=(IMAGE_RES, IMAGE_RES) + (3,), num_classes=102)
#keras.utils.plot_model(model, show_shapes=True)

epochs = 150

#reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0001)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)

model.fit(
    train_batches,
    epochs=epochs,
    validation_data=validation_batches,
    batch_size=BATCH_SIZE,
    validation_batch_size=BATCH_SIZE,
    verbose=1
    #callbacks=[reduce_lr]
)

Epoch 1/150
64/64 [==============================] - 18s 151ms/step - loss: 6.7238 - accuracy: 0.0127 - val_loss: 4.6533 - val_accuracy: 0.0098
Epoch 2/150
64/64 [==============================] - 10s 146ms/step - loss: 5.5239 - accuracy: 0.0098 - val_loss: 4.6970 - val_accuracy: 0.0059
Epoch 3/150
64/64 [==============================] - 9s 137ms/step - loss: 5.1371 - accuracy: 0.0167 - val_loss: 4.7055 - val_accuracy: 0.0098
Epoch 4/150
64/64 [==============================] - 12s 184ms/step - loss: 4.9694 - accuracy: 0.0088 - val_loss: 4.6868 - val_accuracy: 0.0078
Epoch 5/150
64/64 [==============================] - 10s 152ms/step - loss: 4.8977 - accuracy: 0.0078 - val_loss: 4.6630 - val_accuracy: 0.0098
Epoch 6/150
64/64 [==============================] - 9s 135ms/step - loss: 4.8085 - accuracy: 0.0137 - val_loss: 4.6346 - val_accuracy: 0.0078
Epoch 7/150
64/64 [==============================] - 10s 146ms/step - loss: 4.7821 - accuracy: 0.0098 - val_loss: 4.6271 - val_accuracy: 0

KeyboardInterrupt: ignored

In [12]:
model.save('OxfordFlowers102-2.h5')

In [ ]:
test_loss, test_acc = model.evaluate(test_batches, verbose=1, batch_size=BATCH_SIZE)
print("Test accuracy:", test_acc)

193/193 [==============================] - 14s 70ms/step - loss: 3.7169 - accuracy: 0.2558
Test accuracy: 0.25581395626068115


In [ ]:
for features in train_batches.take(1):
  print(features)

(<tf.Tensor: shape=(32, 224, 224, 3), dtype=float32, numpy=
array([[[[2.20325626e-02, 3.77188362e-02, 3.37972678e-02],
         [2.20325626e-02, 3.77188362e-02, 3.37972678e-02],
         [2.20325626e-02, 3.77188362e-02, 3.37972678e-02],
         ...,
         [2.94296108e-02, 2.94296108e-02, 2.94296108e-02],
         [2.35294122e-02, 2.35294122e-02, 2.35294122e-02],
         [2.35294122e-02, 2.35294122e-02, 2.35294122e-02]],

        [[2.35294122e-02, 3.92156877e-02, 3.52941193e-02],
         [2.35294122e-02, 3.92156877e-02, 3.52941193e-02],
         [2.35294122e-02, 3.92156877e-02, 3.52941193e-02],
         ...,
         [2.63661519e-02, 2.63661519e-02, 2.63661519e-02],
         [2.35294122e-02, 2.35294122e-02, 2.35294122e-02],
         [2.35294122e-02, 2.35294122e-02, 2.35294122e-02]],

        [[2.74509806e-02, 4.31372561e-02, 3.92156877e-02],
         [2.74509806e-02, 4.31372561e-02, 3.92156877e-02],
         [2.74509806e-02, 4.31372561e-02, 3.92156877e-02],
         ...,
         